In [1]:
!python --version

Python 3.10.9


In [2]:
!nvidia-smi

Tue Jan 31 22:43:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   65C    P0    40W /  70W |   5583MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [4]:
import torch
torch.__version__

'2.0.0.dev20230125'

In [5]:
import transformers
transformers.__version__

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.22.2'

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch._dynamo as torchdynamo
import torch

In [7]:
# default cache size needs to be increased to store the many graphs with generative models
torchdynamo.config.cache_size_limit = 512

In [9]:
model_name = "google/mt5-small"

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [11]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt", padding='max_length', truncation=True, max_length=50).to('cuda')

# dynamo

In [12]:
model.generate2 = torchdynamo.optimize("inductor")(model.generate)

In [13]:
# dynamo warm up
print(tokenizer.batch_decode(model.generate2(**inputs, min_length=15, max_length=15)))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[2023-01-31 22:44:52,814] torch._inductor.compile_fx: [WARNING] skipping cudagraphs due to input mutation
[2023-01-31 22:45:00,109] torch._inductor.compile_fx: [WARNING] skipping cudagraphs due to input mutation
[2023-01-31 22:45:06,939] torch._inductor.compile_fx: [WARNING] skipping cudagraphs due to input mutation
[2023-01-31 22:45:13,739] torch._inductor.compile_fx: [WARNING] skipping cudagraphs due to input mutation
[2023-01-31 22:45:20,487] torch._inductor.compile_fx: [WARNING] skipping cudagraphs due to input mutation
[2023-01-31 22:45:27,113] torch._inductor.compile_fx: [WARNING] skipping cudagraphs due to input mutation
[2023-01-31 22:45:33,842] torch._inductor.compile_fx: [WARNING] skipping cudagraphs due to input mutation
[2023-01-31 22:45:40,435] torch._inductor.compile_fx: [WARNING] skipping cudagraphs due to input mutation
[2023-01-31 22:45:46,998] torch._inductor.compile_fx: [WARNING] skipping cudagraphs due to input mutation
[2023-01-31 22:45:53,465] torch._inductor.comp

['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']


In [15]:
%%timeit
print(tokenizer.batch_decode(model.generate2(**inputs, min_length=15, max_length=15)))

['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> <extra_id_0>.))))).). <extra_id_49> ) <extra_id_56>']
['<pad> 

In [16]:
inputs2 = tokenizer("Generate taxonomy for query: women gucci", return_tensors="pt", padding='max_length', truncation=True, max_length=50).to('cuda')

In [17]:
%%timeit
print(tokenizer.batch_decode(model.generate2(**inputs2, min_length=15, max_length=15)))

['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_36> gucci gucci gucci gucci gucci']
['<pad> <extra_id_0>.) <extra_id_3

In [18]:
inputs3 = tokenizer("Generate taxonomy for query: baby milk", return_tensors="pt", padding='max_length', truncation=True, max_length=50).to('cuda')

In [19]:
%%timeit
print(tokenizer.batch_decode(model.generate2(**inputs3, min_length=15, max_length=15)))

['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.))))))))))).']
['<pad> <extra_id_0>.)))))))

# constrained

In [20]:
from typing import List, Dict
class Trie(object):
    def __init__(self, sequences: List[List[int]] = []):
        self.trie_dict = {}
        self.len = 0
        if sequences:
            for sequence in sequences:
                Trie._add_to_trie(sequence, self.trie_dict)
                self.len += 1

        self.append_trie = None
        self.bos_token_id = None

    def append(self, trie, bos_token_id):
        self.append_trie = trie
        self.bos_token_id = bos_token_id

    def add(self, sequence: List[int]):
        Trie._add_to_trie(sequence, self.trie_dict)
        self.len += 1

    def get(self, prefix_sequence: List[int]):
        return Trie._get_from_trie(
            prefix_sequence, self.trie_dict, self.append_trie, self.bos_token_id
        )

    @staticmethod
    def load_from_dict(trie_dict):
        trie = Trie()
        trie.trie_dict = trie_dict
        trie.len = sum(1 for _ in trie)
        return trie

    @staticmethod
    def _add_to_trie(sequence: List[int], trie_dict: Dict):
        if sequence:
            if sequence[0] not in trie_dict:
                trie_dict[sequence[0]] = {}
            Trie._add_to_trie(sequence[1:], trie_dict[sequence[0]])

    @staticmethod
    def _get_from_trie(
        prefix_sequence: List[int],
        trie_dict: Dict,
        append_trie=None,
        bos_token_id: int = None,
    ):
        if len(prefix_sequence) == 0:
            output = list(trie_dict.keys())
            if append_trie and bos_token_id in output:
                output.remove(bos_token_id)
                output += list(append_trie.trie_dict.keys())
            return output
        elif prefix_sequence[0] in trie_dict:
            return Trie._get_from_trie(
                prefix_sequence[1:],
                trie_dict[prefix_sequence[0]],
                append_trie,
                bos_token_id,
            )
        else:
            if append_trie:
                return append_trie.get(prefix_sequence)
            else:
                return []

    def __iter__(self):
        def _traverse(prefix_sequence, trie_dict):
            if trie_dict:
                for next_token in trie_dict:
                    yield from _traverse(
                        prefix_sequence + [next_token], trie_dict[next_token]
                    )
            else:
                yield prefix_sequence

        return _traverse([], self.trie_dict)

    def __len__(self):
        return self.len

    def __getitem__(self, value):
        return self.get(value)


In [30]:
allowed_gen_sequences = []
with open('../modelling/datasets/taxonomy/wish_v1.2.1_newtax_leafpaths.txt', 'r') as f:
    for l in f:
        l = l.replace('\n', '').strip()
        if len(l) > 0:
            allowed_gen_sequences.append(l)

# allowed_tokids = [
#     [tokenizer.pad_token_id] + tokenizer.encode(i) + [tokenizer.eos_token_id] for i in allowed_gen_sequences
# ]

allowed_tokids = [
    [tokenizer.pad_token_id] + tokenizer.encode(i) + [tokenizer.eos_token_id] for i in allowed_gen_sequences
]
max_len = max(len(i) for i in allowed_tokids)


In [32]:
trie_fake = Trie([[0] * max_len])
trie = Trie(allowed_tokids)

def constraint_fake(batch_id, sent):
    return trie_fake.get(sent.tolist())
    
def constraint(batch_id, sent):
    return trie.get(sent.tolist())


In [33]:
batch = inputs
batch2 = inputs2
batch3 = inputs3

## constrained beam-search

In [34]:
model.generate3 = torchdynamo.optimize("inductor")(model.generate)

In [35]:
infres = model.generate3(
    input_ids = batch["input_ids"], 
    attention_mask = batch["attention_mask"],
    num_beams = 3, 
    num_return_sequences = 3, 
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint_fake, # use longest fake trie to warm up
    output_scores=True, return_dict_in_generate=True
)
prediction = infres.sequences
probs = infres.sequences_scores.exp()
print(tokenizer.batch_decode(prediction))
print(probs)

[2023-01-31 23:00:08,964] torch._dynamo.symbolic_convert: [WARNING] /tmp/ipykernel_3614/4161888379.py <function constraint_fake at 0x7f87f24e3910> [UserDefinedObjectVariable(PrefixConstrainedLogitsProcessor), ConstantVariable(int), TensorVariable()] {} too many positional arguments


['<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad></s>', '<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad></s><pad>', '<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad></s><pad><pad>']
tensor([0., 0., 0.], device='cuda:0')


In [39]:
%%timeit
infres = model.generate3(
    input_ids = batch["input_ids"], 
    attention_mask = batch["attention_mask"],
    num_beams = 3, 
    num_return_sequences = 3, 
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint, 
    output_scores=True, return_dict_in_generate=True
)
prediction = infres.sequences
probs = infres.sequences_scores.exp()
print(tokenizer.batch_decode(prediction))
print(probs)

['<pad> consumer electronics > power source > batteries > replacement batteries</s><pad><pad>', '<pad> consumer electronics > power source > batteries > button cell batteries</s><pad><pad>', '<pad> consumer electronics > power source > batteries > primary & dry batteries</s>']
tensor([0., 0., 0.], device='cuda:0')
['<pad> consumer electronics > power source > batteries > replacement batteries</s><pad><pad>', '<pad> consumer electronics > power source > batteries > button cell batteries</s><pad><pad>', '<pad> consumer electronics > power source > batteries > primary & dry batteries</s>']
tensor([0., 0., 0.], device='cuda:0')
['<pad> consumer electronics > power source > batteries > replacement batteries</s><pad><pad>', '<pad> consumer electronics > power source > batteries > button cell batteries</s><pad><pad>', '<pad> consumer electronics > power source > batteries > primary & dry batteries</s>']
tensor([0., 0., 0.], device='cuda:0')
['<pad> consumer electronics > power source > batter

In [40]:
%%timeit
infres = model.generate3(
    input_ids = batch2["input_ids"], 
    attention_mask = batch2["attention_mask"],
    num_beams = 3, 
    num_return_sequences = 3, 
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint, 
    output_scores=True, return_dict_in_generate=True
)
prediction = infres.sequences
probs = infres.sequences_scores.exp()
print(tokenizer.batch_decode(prediction))
print(probs)

['<pad> tools > abrasive tools</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>', '<pad> consumer electronics > hifi devices > hifi players</s><pad><pad><pad><pad><pad>', '<pad> consumer electronics > earphones & headphones > earphones</s>']
tensor([1.9244e-40, 0.0000e+00, 0.0000e+00], device='cuda:0')
['<pad> tools > abrasive tools</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>', '<pad> consumer electronics > hifi devices > hifi players</s><pad><pad><pad><pad><pad>', '<pad> consumer electronics > earphones & headphones > earphones</s>']
tensor([1.9244e-40, 0.0000e+00, 0.0000e+00], device='cuda:0')
['<pad> tools > abrasive tools</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>', '<pad> consumer electronics > hifi devices > hifi players</s><pad><pad><pad><pad><pad>', '<pad> consumer electronics > earphones & headphones > earphones</s>']
tensor([1.9244e-40, 0.0000e+00, 0.0000e+00], device='cuda:0')
['<pad> tools > abrasive tools</s><pad><pad

In [43]:
%%timeit
infres = model.generate3(
    input_ids = batch3["input_ids"], 
    attention_mask = batch3["attention_mask"],
    num_beams = 3, 
    num_return_sequences = 3, 
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint, 
    output_scores=True, return_dict_in_generate=True
)
prediction = infres.sequences
probs = infres.sequences_scores.exp()
print(tokenizer.batch_decode(prediction))
print(probs)

['<pad> consumer electronics > power source > batteries > replacement batteries</s><pad><pad>', '<pad> consumer electronics > power source > batteries > digital batteries</s><pad><pad><pad>', '<pad> consumer electronics > power source > batteries > primary & dry batteries</s>']
tensor([0., 0., 0.], device='cuda:0')
['<pad> consumer electronics > power source > batteries > replacement batteries</s><pad><pad>', '<pad> consumer electronics > power source > batteries > digital batteries</s><pad><pad><pad>', '<pad> consumer electronics > power source > batteries > primary & dry batteries</s>']
tensor([0., 0., 0.], device='cuda:0')
['<pad> consumer electronics > power source > batteries > replacement batteries</s><pad><pad>', '<pad> consumer electronics > power source > batteries > digital batteries</s><pad><pad><pad>', '<pad> consumer electronics > power source > batteries > primary & dry batteries</s>']
tensor([0., 0., 0.], device='cuda:0')
['<pad> consumer electronics > power source > bat

## constrained greedy decoding

In [44]:
model.generate4 = torchdynamo.optimize("inductor")(model.generate)

In [45]:
print(tokenizer.batch_decode(model.generate4(
    input_ids = batch["input_ids"], 
    attention_mask = batch["attention_mask"],
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint_fake
)))

['<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']


In [49]:
%%timeit
print(tokenizer.batch_decode(model.generate4(
    input_ids = batch["input_ids"], 
    attention_mask = batch["attention_mask"],
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint
)))

['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphones</s>']
['<pad> consumer electronics > earphones & headphones > earphone

In [50]:
%%timeit
print(tokenizer.batch_decode(model.generate4(
    input_ids = batch2["input_ids"], 
    attention_mask = batch2["attention_mask"],
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint
)))

['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones

In [51]:
%%timeit
print(tokenizer.batch_decode(model.generate4(
    input_ids = batch3["input_ids"], 
    attention_mask = batch3["attention_mask"],
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint
)))

['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones & headphones > headphone/headset</s>']
['<pad> consumer electronics > earphones

In [60]:
import dvc.api
import pandas as pd
from tqdm import tqdm

In [58]:
df_test = pd.read_json(dvc.api.get_url(
    path='datasets/data/query_label/processed/Offshore_Labelled_Query_Classification_Test.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [61]:
for i in tqdm(df_test['query'].tolist()):
    batch_ = tokenizer(f"Generate taxonomy for query: {i}", return_tensors="pt", padding='max_length', truncation=True, max_length=50).to('cuda')
    tokenizer.batch_decode(model.generate4(
        input_ids = batch_["input_ids"], 
        attention_mask = batch_["attention_mask"],
        do_sample = False, 
        length_penalty = 0, 
        max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
        prefix_allowed_tokens_fn=constraint
    ))

100%|██████████| 4300/4300 [05:19<00:00, 13.45it/s]


In [62]:
(5 * 60 + 19) / 4300

0.07418604651162791